In [6]:
from utils import normalize_feats
from utils import test_train_split
from utils import LinearRegression, Metrics
import matplotlib.pyplot as plt, numpy as np, pandas as pd, random

In [31]:
df = pd.read_csv('AmesHousing_NumericalFeatures.csv')

lin_reg = LinearRegression()

X = df.iloc[:,:-1].values
y = df.iloc[:,-1:].values

lin_reg.fit(X, y)

metrics = Metrics(X,y,lin_reg)
metrics.pretty_print_stats()

sse:     781727477820219.0000
sst:     17365298514614.5117
mse:     284574982824.9796
r^2:     -44.0166
adj_r^2: -44.5474


In [32]:
corr_thresh_min = 0.05
target_corr_dict = df.corr().iloc[:,-1].to_dict()
sort_target_corr = dict(sorted(target_corr_dict.items(), key=lambda item: np.abs(item[1])))
high_corr_feats = list({k: v for k,v in sort_target_corr.items() 
                   if np.abs(v) > corr_thresh_min and k != 'SalePrice'}.keys())

high_corr_df = df[high_corr_feats+['SalePrice']]

X = high_corr_df.iloc[:,:-1].values
y = high_corr_df.iloc[:,-1:].values

lin_reg.fit(X, y)

metrics = Metrics(X,y,lin_reg)
metrics.pretty_print_stats()

sse:     3683744841558.4204
sst:     17365298514614.5117
mse:     1341006494.9248
r^2:     0.7879
adj_r^2: 0.7859


In [33]:
cols = list(high_corr_df.columns)
df = pd.DataFrame(columns=cols)
for col in cols:
    df[col] = list(high_corr_df[col].values)
df.head()

,Pool Area,Kitchen AbvGr,Screen Porch,Enclosed Porch,Bedroom AbvGr,Bsmt Unf SF,Lot Area,2nd Flr SF,Half Bath,Bsmt Full Bath,...,Year Remod/Add,Garage Yr Blt,Year Built,Full Bath,1st Flr SF,Total Bsmt SF,Garage Area,Garage Cars,Gr Liv Area,SalePrice
0,0,1,0,0,3,441.0,31770,0,0,1.0,...,1960,1960.0,1960,1,1656,1080.0,528.0,2.0,1656,215000
1,0,1,120,0,2,270.0,11622,0,0,0.0,...,1961,1961.0,1961,1,896,882.0,730.0,1.0,896,105000
2,0,1,0,0,3,406.0,14267,0,1,0.0,...,1958,1958.0,1958,1,1329,1329.0,312.0,1.0,1329,172000
3,0,1,0,0,3,1045.0,11160,0,1,1.0,...,1968,1968.0,1968,2,2110,2110.0,522.0,2.0,2110,244000
4,0,1,0,0,3,137.0,13830,701,1,0.0,...,1998,1997.0,1997,2,928,928.0,482.0,2.0,1629,189900


In [34]:
max_vals_dict = normalize_feats(df)

In [35]:
orig_sale_price_df = pd.read_csv('AmesHousing_NumericalFeatures.csv')
sale_prices = list(orig_sale_price_df['SalePrice'].values)

df.drop(columns=['SalePrice'], inplace=True)
max_log_sale_price = np.max([np.log(sale_price) for sale_price in sale_prices])
df['Log SalePrice'] = [np.log(sale_price)/max_log_sale_price for sale_price in sale_prices]

In [36]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1:].values

lin_reg.fit(X, y)

metrics = Metrics(X,y,lin_reg)
metrics.pretty_print_stats()

sse:     0.4355
sst:     2.2990
mse:     0.0002
r^2:     0.8106
adj_r^2: 0.8088


In [23]:
cols = list(df.columns)
for col in cols:
    vals = list(df[col].values)
    mean_val = np.mean(vals)
    max_val = np.max(vals)
    df[col] = df[col].apply(lambda x: x-mean_val)
df

,Pool Area,Kitchen AbvGr,Screen Porch,Enclosed Porch,Bedroom AbvGr,Bsmt Unf SF,Lot Area,2nd Flr SF,Half Bath,Bsmt Full Bath,...,Year Remod/Add,Garage Yr Blt,Year Built,Full Bath,1st Flr SF,Total Bsmt SF,Garage Area,Garage Cars,Gr Liv Area,Log SalePrice
0,-0.002991,-0.011649,-0.029425,-0.021814,0.025543,-0.050259,0.099893,-0.164016,-0.193484,0.28067,...,-0.012281,-0.008893,-0.006320,-0.143611,0.095260,0.002283,0.019388,0.027011,0.025328,0.017038
1,-0.002991,-0.011649,0.178909,-0.021814,-0.141124,-0.123461,0.006288,-0.164016,-0.193484,-0.21933,...,-0.011783,-0.008395,-0.005823,-0.143611,-0.053906,-0.030122,0.155140,-0.172989,-0.109376,-0.035914
2,-0.002991,-0.011649,-0.029425,-0.021814,0.025543,-0.065242,0.018577,-0.164016,0.306516,-0.21933,...,-0.013276,-0.009888,-0.007315,-0.143611,0.031079,0.043036,-0.125774,-0.172989,-0.032630,0.000551
3,-0.002991,-0.011649,-0.029425,-0.021814,0.025543,0.208302,0.004142,-0.164016,0.306516,0.28067,...,-0.008301,-0.004912,-0.002340,0.106389,0.184367,0.170860,0.015355,0.027011,0.105796,0.026386
4,-0.002991,-0.011649,-0.029425,-0.021814,0.025543,-0.180396,0.016546,0.175452,0.306516,-0.21933,...,0.006625,0.009515,0.012088,0.106389,-0.047625,-0.022594,-0.011526,0.027011,0.020543,0.007865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,-0.002991,-0.011649,-0.029425,-0.021814,0.192210,-0.239044,0.045211,-0.164016,-0.193484,0.28067,...,0.005630,-0.008893,-0.006320,-0.143611,0.010471,0.025851,0.051646,0.027011,-0.051240,-0.019568
2743,-0.002991,-0.011649,-0.029425,-0.021814,0.025543,-0.160276,-0.010832,-0.164016,-0.193484,0.28067,...,-0.000340,0.003048,0.005620,-0.143611,-0.032905,-0.010319,0.059710,0.027011,-0.090411,-0.013351
2744,-0.002991,-0.011649,-0.029425,-0.021814,-0.141124,-0.136732,-0.006428,-0.164016,-0.193484,0.28067,...,-0.000838,0.002550,0.005122,-0.143611,-0.052728,-0.033068,-0.010182,0.027011,-0.108312,-0.019568
2745,-0.002991,-0.011649,-0.029425,-0.021814,-0.141124,-0.155568,-0.001201,-0.164016,-0.193484,0.28067,...,-0.004818,-0.001430,0.000645,-0.143611,0.042856,0.052856,-0.054537,0.027011,-0.021995,-0.000314


In [37]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1:].values

lin_reg.fit(X, y)

metrics = Metrics(X,y,lin_reg)
metrics.pretty_print_stats()

sse:     0.4355
sst:     2.2990
mse:     0.0002
r^2:     0.8106
adj_r^2: 0.8088


In [38]:
lin_reg = LinearRegression()

In [41]:
unseen_data_r_sqs = list()
for i in range(100):
    
    X_train, y_train, X_test, y_test = test_train_split(df, 0.2)
    lin_reg.fit(X_train, y_train)
    metrics = Metrics(X_test, y_test, lin_reg)
    unseen_data_r_sqs.append(metrics.r_sq())

In [42]:
np.mean(unseen_data_r_sqs)

0.7971113253744829

In [43]:
np.std(unseen_data_r_sqs)

0.05539865166821743

In [44]:
df

,Pool Area,Kitchen AbvGr,Screen Porch,Enclosed Porch,Bedroom AbvGr,Bsmt Unf SF,Lot Area,2nd Flr SF,Half Bath,Bsmt Full Bath,...,Year Remod/Add,Garage Yr Blt,Year Built,Full Bath,1st Flr SF,Total Bsmt SF,Garage Area,Garage Cars,Gr Liv Area,Log SalePrice
0,0.0,0.333333,0.000000,0.0,0.500000,0.188784,0.147599,0.000000,0.0,0.5,...,0.975124,0.975124,0.975124,0.25,0.325025,0.176759,0.354839,0.4,0.293513,0.907194
1,0.0,0.333333,0.208333,0.0,0.333333,0.115582,0.053994,0.000000,0.0,0.0,...,0.975622,0.975622,0.975622,0.25,0.175859,0.144354,0.490591,0.2,0.158809,0.854242
2,0.0,0.333333,0.000000,0.0,0.500000,0.173801,0.066283,0.000000,0.5,0.0,...,0.974129,0.974129,0.974129,0.25,0.260844,0.217512,0.209677,0.2,0.235555,0.890707
3,0.0,0.333333,0.000000,0.0,0.500000,0.447346,0.051848,0.000000,0.5,0.5,...,0.979104,0.979104,0.979104,0.50,0.414132,0.345336,0.350806,0.4,0.373981,0.916543
4,0.0,0.333333,0.000000,0.0,0.500000,0.058647,0.064252,0.339467,0.5,0.0,...,0.994030,0.993532,0.993532,0.50,0.182139,0.151882,0.323925,0.4,0.288727,0.898022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,0.0,0.333333,0.000000,0.0,0.666667,0.000000,0.092917,0.000000,0.0,0.5,...,0.993035,0.975124,0.975124,0.25,0.240236,0.200327,0.387097,0.4,0.216944,0.870588
2743,0.0,0.333333,0.000000,0.0,0.500000,0.078767,0.036874,0.000000,0.0,0.5,...,0.987065,0.987065,0.987065,0.25,0.196860,0.164157,0.395161,0.4,0.177774,0.876805
2744,0.0,0.333333,0.000000,0.0,0.333333,0.102312,0.041279,0.000000,0.0,0.5,...,0.986567,0.986567,0.986567,0.25,0.177036,0.141408,0.325269,0.4,0.159872,0.870588
2745,0.0,0.333333,0.000000,0.0,0.333333,0.083476,0.046505,0.000000,0.0,0.5,...,0.982587,0.982587,0.982090,0.25,0.272620,0.227332,0.280914,0.4,0.246189,0.889843


In [46]:
df.to_csv('AmesHousing_NormNumericalFeatures.csv', index=False)